In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from spot.utils import cst, read_file, write_file, SpecialNames, proj_root
import os
from spot.type_checking import collect_annotations, MypyChecker, AnnotPath, mypy_checker
from spot.type_inference import TypeInfEnv, TypeInfAction
import shutil


if os.getcwd().endswith('scripts'):
    os.chdir("..")

In [6]:
parsed = cst.parse_module(read_file("data/code/good_code_1.py"))
collect_annotations(parsed)

([AnnotPath('x_str'),
  AnnotPath('y'),
  AnnotPath('z_str'),
  AnnotPath('Foo.__init__.x'),
  AnnotPath('Foo.__init__.self.x'),
  AnnotPath('Foo.__init__.self.y'),
  AnnotPath('Foo.__init__.<return>'),
  AnnotPath('Foo.foo.z'),
  AnnotPath('Foo.foo.<return>')],
 {AnnotPath('x_str'): Annotation(
      annotation=Name(
          value='str',
          lpar=[],
          rpar=[],
      ),
      whitespace_before_indicator=SimpleWhitespace(
          value='',
      ),
      whitespace_after_indicator=SimpleWhitespace(
          value=' ',
      ),
  ),
  AnnotPath('y'): Annotation(
      annotation=Name(
          value='Any',
          lpar=[],
          rpar=[],
      ),
      whitespace_before_indicator=SimpleWhitespace(
          value='',
      ),
      whitespace_after_indicator=SimpleWhitespace(
          value=' ',
      ),
  ),
  AnnotPath('z_str'): Annotation(
      annotation=Name(
          value='str',
          lpar=[],
          rpar=[],
      ),
      whitespace_before_in

In [8]:
print(parsed)

Module(
    body=[
        SimpleStatementLine(
            body=[
                ImportFrom(
                    module=Name(
                        value='typing',
                        lpar=[],
                        rpar=[],
                    ),
                    names=[
                        ImportAlias(
                            name=Name(
                                value='Any',
                                lpar=[],
                                rpar=[],
                            ),
                            asname=None,
                            comma=MaybeSentinel.DEFAULT,
                        ),
                    ],
                    relative=[],
                    lpar=None,
                    rpar=None,
                    semicolon=MaybeSentinel.DEFAULT,
                    whitespace_after_from=SimpleWhitespace(
                        value=' ',
                    ),
                    whitespace_before_import=SimpleWhitespace(
    

In [12]:
with mypy_checker(".venv/bin/dmypy", "data/code") as checker:
    check_r = checker.check_code_dir()
    print(check_r.num_error_dict)

Daemon started
{'bad_code_1.py': 7, 'bad_code_2.py': 1}
Daemon stopped


In [13]:
# remove `data/temp` if it exists
inference_dir = "data/code_output/inference"
if os.path.exists(inference_dir):
    shutil.rmtree(inference_dir)
if not os.path.exists(inference_dir):
    os.mkdir(inference_dir)
write_file(f"{inference_dir}/env_code_1.py", read_file("data/code/env_code_1.py"))

In [14]:
inf_checker = MypyChecker(".venv/bin/dmypy", inference_dir)
env = TypeInfEnv(inf_checker, f"{inference_dir}/env_code_1.py")
print(env.state)

Daemon started

num_errors: 0
num_to_annot: 11
to_annotate: [AnnotPath('n'), AnnotPath('<return>'), AnnotPath('bar'), AnnotPath('<return>'), AnnotPath('a'), AnnotPath('b'), AnnotPath('<return>'), AnnotPath('a'), AnnotPath('b'), AnnotPath('c'), AnnotPath('<return>')]
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b + "c"

def int_tripple_add(a, b, c):
    return a + b + c
        


In [15]:
env.step(TypeInfAction(env.state.to_annot[0], cst.Name("str")))
print(env.state)


num_errors: 0
num_to_annot: 10
to_annotate: [AnnotPath('<return>'), AnnotPath('bar'), AnnotPath('<return>'), AnnotPath('a'), AnnotPath('b'), AnnotPath('<return>'), AnnotPath('a'), AnnotPath('b'), AnnotPath('c'), AnnotPath('<return>')]
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b + "c"

def int_tripple_add(a, b, c):
    return a + b + c
        


In [16]:
env.reset()

In [17]:
inf_checker.recheck_files("env_code_1.py")

MypyResult(num_errors=0, num_error_dict={}, output_str='Success: no issues found in 2 source files\n')

In [18]:
env.step(TypeInfAction(env.state.to_annot[0], cst.Name("str")))
print(env.state)


num_errors: 0
num_to_annot: 10
to_annotate: [AnnotPath('<return>'), AnnotPath('bar'), AnnotPath('<return>'), AnnotPath('a'), AnnotPath('b'), AnnotPath('<return>'), AnnotPath('a'), AnnotPath('b'), AnnotPath('c'), AnnotPath('<return>')]
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b + "c"

def int_tripple_add(a, b, c):
    return a + b + c
        


In [19]:
env.reset()

while len(env.state.to_annot) > 0:
    env.step(TypeInfAction(env.state.to_annot[0], cst.Name("str")))

print(env.state)


num_errors: 0
num_to_annot: 0
to_annotate: []
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b + "c"

def int_tripple_add(a, b, c):
    return a + b + c
        


In [20]:
env.reset()

while len(env.state.to_annot) > 0:
    env.step(TypeInfAction(env.state.to_annot[0], cst.Name("int")))

print(env.state)


num_errors: 0
num_to_annot: 0
to_annotate: []
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b + "c"

def int_tripple_add(a, b, c):
    return a + b + c
        


In [21]:
import random

env.reset()

while len(env.state.to_annot) > 0:
    ty = cst.Name(random.choice(["int", "str"]))
    env.step(TypeInfAction(env.state.to_annot[0], ty))

print(env.state)


num_errors: 0
num_to_annot: 0
to_annotate: []
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b + "c"

def int_tripple_add(a, b, c):
    return a + b + c
        
